In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
#for dirname, _, filenames in os.walk('/kaggle/input'):
#    for filename in filenames:
#        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import StratifiedShuffleSplit
from keras.models import Sequential
from keras.layers import Dense, Activation, Flatten, Convolution1D, Dropout
# from keras.optimizers import SGD
from tensorflow.keras.optimizers import SGD
from keras.utils import np_utils

import itertools, time, datetime


In [3]:
train = pd.read_csv('Datasets/Dataset1_Training.csv')
test = pd.read_csv('Datasets/Dataset1_Testing.csv')

#train = pd.read_csv('Datasets/Dataset2_Training.csv')
#test = pd.read_csv('Datasets/Dataset2_Testing.csv')

In [4]:
def encode(train, test):
    label_encoder = LabelEncoder().fit(train.species)
    labels = label_encoder.transform(train.species)
    classes = list(label_encoder.classes_)

    train = train.drop(['species', 'id'], axis=1)
    test_ids=test.id
    test = test.drop('id', axis=1)

    return train, labels, test, classes,test_ids

In [5]:
train, labels, test, classes,test_ids = encode(train, test)


# standardize train features

In [6]:
scaler = StandardScaler().fit(train.values)
scaled_train = scaler.transform(train.values)

# split train data into train and validation

In [7]:
sss = StratifiedShuffleSplit(test_size=0.1, random_state=23)
for train_index, valid_index in sss.split(scaled_train, labels):
    X_train, X_valid = scaled_train[train_index], scaled_train[valid_index]
    y_train, y_valid = labels[train_index], labels[valid_index]
    

In [8]:
nb_features = 64 # number of features per features type (shape, texture, margin)   
nb_class = len(classes)

# reshape

In [9]:
# reshape train data
X_train_r = np.zeros((len(X_train), nb_features, 3))
X_train_r[:, :, 0] = X_train[:, :nb_features]
X_train_r[:, :, 1] = X_train[:, nb_features:128]
X_train_r[:, :, 2] = X_train[:, 128:]

# reshape validation data
X_valid_r = np.zeros((len(X_valid), nb_features, 3))
X_valid_r[:, :, 0] = X_valid[:, :nb_features]
X_valid_r[:, :, 1] = X_valid[:, nb_features:128]
X_valid_r[:, :, 2] = X_valid[:, 128:]


# Timer

In [1]:
# Timer 

# Utility custom contextual class for calculating 
# the time taken for a certain code block to execute
    
class CodeTimer:
    
    def __init__(self, name=None):
        self.name = " '"  + name + "'" if name else ''

    def __enter__(self):
        self.start = time.perf_counter()

    def __exit__(self, exc_type, exc_value, traceback):
        self.took = (time.perf_counter() - self.start) * 1000.0
        time_taken = datetime.timedelta(milliseconds = self.took)
        print('Code block' + self.name + ' took(HH:MM:SS): ' + str(time_taken))

# Keras model 

In [11]:
# Keras model with one Convolution1D layer
# unfortunately more number of covnolutional layers, filters and filters lenght 
# don't give better accuracy
model = Sequential()
model.add(Convolution1D(512, 1, input_shape=(nb_features, 3)))
model.add(Activation('relu'))
model.add(Flatten())
model.add(Dropout(0.4))
model.add(Dense(2048, activation='relu'))
model.add(Dense(1024, activation='relu'))
model.add(Dense(nb_class))
model.add(Activation('softmax'))

2022-11-07 19:59:41.895376: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [12]:

y_train = np_utils.to_categorical(y_train, nb_class)
y_valid = np_utils.to_categorical(y_valid, nb_class)

sgd = SGD(lr=0.01, nesterov=True, decay=1e-6, momentum=0.9)
model.compile(loss='categorical_crossentropy',optimizer=sgd,metrics=['accuracy'])

# define an object for training timer
training_timer = CodeTimer('training')

nb_epoch = 15
# calculate training time
with training_timer:
    model.fit(X_train_r, y_train, epochs=nb_epoch, validation_data=(X_valid_r, y_valid), batch_size=16)

/Users/muneerahmohammad/opt/anaconda3/lib/python3.8/site-packages/keras/optimizer_v2/optimizer_v2.py:355: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  warnings.warn(
2022-11-07 19:59:43.866176: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


Epoch 1/15
90/90 [==============================] - 26s 283ms/step - loss: 2.9094 - accuracy: 0.3579 - val_loss: 1.0829 - val_accuracy: 0.7250
Epoch 2/15
90/90 [==============================] - 25s 278ms/step - loss: 0.5410 - accuracy: 0.8589 - val_loss: 0.4052 - val_accuracy: 0.8875
Epoch 3/15
90/90 [==============================] - 25s 281ms/step - loss: 0.2103 - accuracy: 0.9395 - val_loss: 0.3779 - val_accuracy: 0.8938
Epoch 4/15
90/90 [==============================] - 25s 282ms/step - loss: 0.1230 - accuracy: 0.9694 - val_loss: 0.2857 - val_accuracy: 0.9375
Epoch 5/15
90/90 [==============================] - 25s 280ms/step - loss: 0.0749 - accuracy: 0.9792 - val_loss: 0.2452 - val_accuracy: 0.9312
Epoch 6/15
90/90 [==============================] - 26s 290ms/step - loss: 0.0155 - accuracy: 0.9979 - val_loss: 0.2274 - val_accuracy: 0.9312
Epoch 7/15
90/90 [==============================] - 26s 286ms/step - loss: 0.0105 - accuracy: 0.9979 - val_loss: 0.2024 - val_accuracy: 0.9375

In [14]:
scaler = StandardScaler().fit(test.values)
scaled_test = scaler.transform(test.values)

In [15]:
test_dataset = np.zeros((len(scaled_test), nb_features, 3))
test_dataset[:, :, 0] = scaled_test[:, :nb_features]
test_dataset[:, :, 1] = scaled_test[:, nb_features:128]
test_dataset[:, :, 2] = scaled_test[:, 128:]

In [16]:
# define an object for testing timer 
testing_timer = CodeTimer('testing')

# calculate testing time
with testing_timer:
    preds_test = model.predict(test_dataset)
    preds_test

Code block 'testing' took(HH:MM:SS): 0:00:00.260288


In [34]:
submission = pd.DataFrame(preds_test, columns=classes)
submission.insert(0, 'id', test_ids)
submission

,id,Acer_Capillipes,Acer_Circinatum,Acer_Mono,Acer_Opalus,Acer_Palmatum,Acer_Pictum,Acer_Platanoids,Acer_Rubrum,Acer_Rufinerve,...,Salix_Fragilis,Salix_Intergra,Sorbus_Aria,Tilia_Oliveri,Tilia_Platyphyllos,Tilia_Tomentosa,Ulmus_Bergmanniana,Viburnum_Tinus,Viburnum_x_Rhytidophylloides,Zelkova_Serrata
0,4,4.627868e-08,6.634896e-08,3.522151e-10,1.979945e-07,5.803275e-08,1.861704e-07,1.154325e-08,6.933443e-09,2.948224e-08,...,9.980554e-10,2.880373e-08,1.176069e-08,1.781970e-09,1.447084e-07,2.871864e-09,1.111443e-08,2.241541e-10,2.935469e-07,1.674585e-08
1,7,7.428685e-09,2.938926e-08,7.177974e-08,2.458765e-05,1.037744e-09,5.196398e-08,3.845309e-05,9.520064e-09,2.419575e-08,...,9.030985e-09,4.874698e-08,7.610932e-09,1.237863e-10,7.649972e-11,7.480708e-06,2.197336e-09,1.024054e-06,2.632089e-09,3.450862e-08
2,9,8.191396e-08,9.985400e-01,1.399405e-09,3.588255e-08,1.199303e-04,4.966346e-07,1.305243e-08,6.328683e-07,2.961180e-05,...,1.644425e-07,6.634397e-10,3.506502e-09,1.220369e-08,2.954225e-08,1.629953e-07,2.807964e-07,2.491332e-10,2.232702e-09,4.196976e-04
3,12,1.150263e-06,2.886420e-02,8.764577e-08,1.001434e-06,1.765437e-07,5.787755e-07,1.526826e-04,2.277724e-06,6.339775e-05,...,4.053422e-07,3.576528e-09,2.353166e-06,8.379025e-08,3.034356e-07,9.784295e-05,2.959457e-03,2.460465e-08,9.947771e-07,3.574744e-03
4,13,1.502209e-08,1.522091e-07,3.322889e-12,8.429778e-10,4.163709e-10,1.364373e-10,1.179851e-08,9.494687e-09,1.127881e-05,...,1.750842e-08,2.094141e-12,1.332169e-08,3.254024e-11,4.863334e-07,2.077794e-07,2.510697e-06,1.644894e-10,4.989625e-09,2.808493e-09
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
589,1576,1.980781e-07,9.992300e-01,1.377305e-09,2.480634e-08,1.132538e-05,1.153638e-07,5.831625e-09,2.102547e-07,1.737823e-05,...,1.027708e-07,4.041034e-10,5.084368e-09,4.577399e-09,1.080938e-08,2.070408e-07,1.218631e-06,2.544389e-10,5.468262e-09,2.023965e-04
590,1577,4.944513e-06,2.319805e-07,1.970380e-09,6.093211e-06,9.794660e-09,7.245751e-09,8.901698e-07,2.040376e-06,1.286026e-04,...,1.959674e-07,2.953459e-09,4.644566e-05,1.333897e-07,7.473868e-05,8.508134e-05,1.242911e-06,4.974089e-09,1.714420e-07,3.797790e-05
591,1579,1.505680e-06,1.657436e-07,2.288140e-09,9.277619e-10,9.284466e-07,1.009479e-06,1.357591e-08,4.307246e-08,1.059829e-08,...,1.816050e-10,1.863703e-10,2.407928e-07,7.499912e-07,9.411053e-07,2.690780e-10,1.739623e-09,2.757382e-09,1.663812e-07,1.342949e-06
592,1580,1.126535e-11,1.447198e-10,2.062197e-08,1.178442e-09,2.203069e-09,2.306448e-11,8.678703e-10,1.264675e-08,3.107717e-11,...,6.001701e-09,8.752934e-09,6.873060e-13,7.803693e-09,3.993096e-12,1.915356e-09,9.416861e-12,1.363226e-09,3.964289e-12,1.397565e-09


In [35]:
submission.to_csv('Resultsubmission.csv', index=False)
print('done!')

done!
